In [1]:
#Importando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
#Baixando os dados disponibilizados pela Loggi (git)
!wget https://github.com/vlsantos-bit/Desafio_Tec_Leitor_Cod_Loggi/archive/refs/heads/main.zip

--2021-06-27 17:36:27--  https://github.com/vlsantos-bit/Desafio_Tec_Leitor_Cod_Loggi/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vlsantos-bit/Desafio_Tec_Leitor_Cod_Loggi/zip/refs/heads/main [following]
--2021-06-27 17:36:27--  https://codeload.github.com/vlsantos-bit/Desafio_Tec_Leitor_Cod_Loggi/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1155 (1.1K) [application/zip]
Saving to: ‘main.zip.1’

main.zip.1          100%[===================>]   1.13K  --.-KB/s    in 0s      

2021-06-27 17:36:27 (16.7 MB/s) - ‘main.zip.1’ saved [1155/1155]



In [3]:
#Descompactando zip
!unzip main.zip 

Archive:  main.zip
77dbe12bb58618ebba7df59bb60bf9ba9fd6f31e
replace Desafio_Tec_Leitor_Cod_Loggi-main/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: Desafio_Tec_Leitor_Cod_Loggi-main/README.md  
  inflating: Desafio_Tec_Leitor_Cod_Loggi-main/dados.csv  
 extracting: Desafio_Tec_Leitor_Cod_Loggi-main/dados.rar  


In [4]:
#Abrindo e renomeando os dados
leg = ['pacotes','cod']
da = pd.read_csv('Desafio_Tec_Leitor_Cod_Loggi-main/dados.csv', sep=';', names=leg)

In [5]:
#Dicionários de consulta
loca = {'111':'Centro-oeste','333':'Nordeste','555':'Norte','888':'Sudeste','000':'Sul'}
obj = {'000':'joias','111':'livro','333':'eletronico','555':'bebidas','888':'brinquedos'}

In [6]:
#a) Identificar o destino de cada pacote.
def local(dado): 
    da_t = dado
    df = da_t['cod']
    nd=[]
    for i in range(0,3,1):
        nd.append("{:,}".format(df[i]))
    for i in range(3,6,1):
        nd.append("{:,}".format(df[i]))
    for i in range(6,9,1):
        nd.append("{:,}".format(df[i]))
    for i in range(9,12,1):
        nd.append("{:,}".format(df[i]))
    for i in range(12,15,1):
        nd.append("{:,}".format(df[i]))
    nd=pd.DataFrame(nd,columns=['cod1'])
    da1 = da.join(nd)
    df2 = da1.assign(names=da1.cod1.str.split(","))
    df2=df2.names.apply(pd.Series)
    da_d = da1.join(df2)
    da_d.columns = ['pacotes','cod','cod1','origem','destino','cod_logg','cod_vend','produto']
    da_d=da_d.drop(columns=['cod','cod_logg','cod_vend','produto'])
    da_d=da_d.replace({"destino": loca})
    print("Destinos")
    return da_d

In [7]:
#Identificar o destino de cada pacote.

local(da)

Destinos


,pacotes,cod1,origem,destino
0,Pacote1,"888,555,555,123,888",888,Norte
1,Pacote2,"333,333,555,584,333",333,Nordeste
2,Pacote3,"222,333,555,124,000",222,Nordeste
3,Pacote4,"111,555,874,555",111,Norte
4,Pacote5,"111,888,555,654,777",111,Sudeste
5,Pacote6,"111,333,555,123,333",111,Nordeste
6,Pacote7,"555,555,555,123,888",555,Norte
7,Pacote8,"888,333,555,584,333",888,Nordeste
8,Pacote9,"111,333,555,124,000",111,Nordeste
9,Pacote10,"333,888,555,584,333",333,Sudeste


In [8]:
#Função para ajustar os dados validos
def ajust(dado):
    #Removendo conjunto fora da Loggi
    da_t = dado.drop(dado[dado.cod < 999999999999].index).reset_index(drop=True)
    df = da_t['cod']
    nd=[]
    for i in range(0,3,1):
        nd.append("{:,}".format(df[i]))
    for i in range(3,6,1):
        nd.append("{:,}".format(df[i]))
    for i in range(6,9,1):
        nd.append("{:,}".format(df[i]))
    for i in range(9,12,1):
        nd.append("{:,}".format(df[i]))
    for i in range(12,13,1):
        nd.append("{:,}".format(df[i]))
    nd=pd.DataFrame(nd,columns=['cod1'])
    da1 = da_t.join(nd)
    df2 = da1.assign(names=da1.cod1.str.split(","))
    df2=df2.names.apply(pd.Series)
    da_d = da1.join(df2)
    #Separando código
    da_d.columns = ['pacotes','cod','cod1','origem','destino','cod_logg','cod_vend','produto']
    da_d=da_d.drop(columns=['cod','cod1'])
    #Aplicando considerações e verificações:
    #1)Removendo vendedor com  CNPJ inativo
    da_d = da_d[da_d.cod_vend != '584']
    da_d.reset_index(drop=True, inplace=True)
    #2)Removendo Origem Centro-oeste(joias)
    da_d = da_d[~((da_d['origem'] == '111') & (da_d['produto'] == '000'))]
    da_d.reset_index(drop=True, inplace=True)
    #Ajuste para facilitar a visualização da tabela 
    da_d=da_d.replace({"destino": loca})
    da_d=da_d.replace({"origem": loca})
    da_d=da_d.replace({"produto": obj})
    #3)Removendo produtos que não estão na lista
    produt_v = ['brinquedos','joias','eletronico','bebidas','Livros']
    da_d=da_d[da_d.produto.isin(produt_v)]
    da_d.reset_index(drop=True, inplace=True)
    #4)Vericando se origem consta na tabela
    orig = ['Sudeste','Norte','Nordeste','Centro-oeste','Sul']
    da_d=da_d[da_d.origem.isin(orig)]
    da_d.reset_index(drop=True, inplace=True)
    print("Pacotes Válidos")
    return da_d

In [9]:
#b) Saber quais pacotes possuem códigos de barras válidos e/ou inválidos.
r_dado=ajust(da)
r_dado

Pacotes Válidos


,pacotes,origem,destino,cod_logg,cod_vend,produto
0,Pacote1,Sudeste,Norte,555,123,brinquedos
1,Pacote6,Centro-oeste,Nordeste,555,123,eletronico
2,Pacote7,Norte,Norte,555,123,brinquedos
3,Pacote11,Norte,Sudeste,555,123,joias
4,Pacote12,Centro-oeste,Sudeste,555,123,bebidas
5,Pacote13,Sudeste,Sul,555,845,eletronico
6,Pacote15,Centro-oeste,Nordeste,555,123,bebidas


In [19]:
r_dado=ajust(da)
#c) Identificar se algum pacote que tem como origem a região Sul tem Brinquedos em seu conteúdo.
#Mudar parametrizações, ex:Origem==produto
r_dado=r_dado['origem'].str.contains('Sul') & r_dado['produto'].str.contains('brinquedos')
if r_dado.any()>0:
    print("Produto indentificado")
else:
    print("Produto não indentificado na região selecionada")


Pacotes Válidos
Produto não indentificado na região selecionada


In [13]:
#d)Listar os pacotes agrupados por região de destino (Considere apenas pacotes válidos)
r_dado=ajust(da)
r_dado=r_dado.drop(columns=['origem','cod_logg','cod_vend','produto'])
test=r_dado.groupby(by=["destino"]).sum()
test

Pacotes Válidos


,pacotes
destino,
Nordeste,Pacote6Pacote15
Norte,Pacote1Pacote7
Sudeste,Pacote11Pacote12
Sul,Pacote13


In [14]:
#e)Listar o número de pacotes enviados por cada vendedor (Considere apenas pacotes válidos).
r_dado=ajust(da)
r_dado=r_dado.drop(columns=['origem','cod_logg','produto','destino'])
test=r_dado.groupby(by=["cod_vend"]).count().reset_index()
print('Pacotes enviados por cada vendendor')
test

Pacotes Válidos
Pacotes enviados por cada vendendor


,cod_vend,pacotes
0,123,6
1,845,1


In [15]:
#Gerar o relatório/lista de pacotes por destino e por tipo (Considere apenas pacotes válidos).
r_dado=ajust(da)
r_dado=r_dado.drop(columns=['origem','cod_logg','cod_vend'])
r_dado

Pacotes Válidos


,pacotes,destino,produto
0,Pacote1,Norte,brinquedos
1,Pacote6,Nordeste,eletronico
2,Pacote7,Norte,brinquedos
3,Pacote11,Sudeste,joias
4,Pacote12,Sudeste,bebidas
5,Pacote13,Sul,eletronico
6,Pacote15,Nordeste,bebidas
